# Api Gateway with Cognito Authentication

 ### Create User
 
 **Copy your App client id** from Cognito User Pools console and set the **clientId** variable
 
 **note:** the user will need to be confirmed either by email or through the cognito console.
 **Set:**
 
 Username
 
 Password
 
 Email

In [ ]:
import json, boto3, requests, datetime

# Client ID goes here
clientId = ''

client = boto3.client('cognito-idp')

signup_response = client.sign_up(
    ClientId=clientId,
    Username='',
    Password='',
    UserAttributes=[{'Name':'email','Value':'email goes here'}])

print(signup_response)

### Login
**note:** Make sure to confirm the user

In [ ]:
auth_response = client.initiate_auth(
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME' : '',
        'PASSWORD' : ''
    },
    ClientId=clientId
)

print(auth_response['AuthenticationResult'].keys())

### Call API with authentication header and token

#### Set the gwid to the gateway id

In [ ]:
#Set gateway id
gwid = ''

headers = {'Authorization': auth_response['AuthenticationResult']['IdToken']}
x = datetime.datetime.now()
orderid = '150'
accountid = 'p150'
vendorid = 'Coffee One'
orderdate = str(x)
coffeetype = 'Short Black'
coffeesize = 'Small'
unitprice = '4.50'
quantity = '1'

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

requests.post(url,json={'order':{
            'orderid': orderid,
            'accountid': accountid,
            'vendorid': vendorid,
            'orderdate':orderdate,
            'details':{
                'coffeetype': coffeetype,
                'coffeesize': coffeesize,
                'unitprice': unitprice,
                'quantity': quantity
            }
        }
    },headers=headers)

response_get = requests.get(url, params={'orderid':150})

print(response_get.json())